<a href="https://colab.research.google.com/github/FilipSagalara/MasterThesis_SNN_frameworks/blob/main/BRAIN2_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST - Spiking neural network with spike timing dependent plasticity

The purpose of this work is to demonstrate local learning rules (STDP) whit the MNIST handwritten digit classification problem. The main ideas are taken from the article [Unsupervised learning of digit recognition using spike-timing-dependent plasticity](https://www.frontiersin.org/articles/10.3389/fncom.2015.00099/full), but the experiment was reconstructed with significant changes from the original work.

## Spiking neural network architecture

To model neuron dynamics leaky integrate-and-fire model is used. Synapses are modeled by conductance changes, i.e., synapses increase their conductance instantaneously by the synaptic weight when a presynaptic spike arrives at the synapse, otherwise the conductance is decaying exponentially.

The network consists of two layers. The first layer is the input layer, containing 28 × 28 neurons (one neuron per image pixel), and the second layer is the processing layer, containing 100 excitatory neurons and 100 inhibitory neurons. Each input is a Poisson spike-train, which is fed to the excitatory neurons of the second layer for the 350 ms following by 150 ms resting period. The rates of each neuron are proportional (devided by 4) to the intensity of the corresponding pixel in the example image.

The excitatory neurons of the second layer are connected in a one-to-one fashion to inhibitory neurons, i.e., each spike in an excitatory neuron will trigger a spike in its corresponding inhibitory neuron. Each of the inhibitory neurons is connected to all excitatory ones, except for the one from which it receives a connection. This connectivity provides lateral inhibition and leads to competition among excitatory neurons. All synapses from input neurons to excitatory neurons are learned using STDP. The weights of the remaining synapses are fixed.

![image.png](attachment:ad6a31c8-9ff4-4e0c-b688-532c1cb8bb43.png)

To understand this part I recomend [https://www.coursera.org/learn/synapses/](https://www.coursera.org/learn/synapses/)

## Experiment design and observed results

1. Feed train set to SNN described above
2. Freeze STDP
3. Feed train set to SNN again and collect generated features (spike counts foreach excitatory neuron during single image representation)
4. Train RandomForest on the top of these features and labels provided
5. Feed test set to SNN and collect new features
6. Predict labels with RandomForest and calculate accuacy score

It reaches 0.9 accuracy on the test set with only 5k training examples and 3 classes. In order to understand whether local learning affects the result, the experiment was repeated but without training SNN with STDP. Second experiment scored 0.74, which clearly shows the efficiency of local training.

In [1]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from google.colab import drive
import pandas as pd

drive.mount('/content/drive/')

class Metrics:

  def __init__(self):
    self.accuracies = []
    self.losses = []
    self.prec_micros = []
    self.rec_micro = []
    self.f1_s_micro = []
    self.prec_macro = []
    self.rec_macro = []
    self.f1_s_macro = []
    self.prec_weight = []
    self.rec_weight = []
    self.f1_s_weight = []
    self.classifi_report = []
    self.confusion_matrices = []
    self.times_per_epoch = []


  def reset(self):
    self.accuracies = []
    self.losses = []
    self.prec_micros = []
    self.rec_micro = []
    self.f1_s_micro = []
    self.prec_macro = []
    self.rec_macro = []
    self.f1_s_macro = []
    self.prec_weight = []
    self.rec_weight = []
    self.f1_s_weight = []
    self.classifi_report = []
    self.confusion_matrices = []
    self.times_per_epoch = []



  
  def my_custom_loss_func(y_true, y_pred):
    diff = np.abs(y_true - y_pred).max()
    return np.log1p(diff)



  def compute_metrics(self, y_pred, y_test, show = False): #y test refers to true labels
    target_names = ['0','1','2','3','4','5','6','7','8','9']
    #Compute loss
    # loss = log_loss(y_test, y_pred, labels = target_names)
    loss = my_custom_loss_func(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)


    prec_micro = precision_score(y_test, y_pred, average='micro')
    rec_micro = recall_score(y_test, y_pred, average='micro')
    f1_s_micro = f1_score(y_test, y_pred, average='micro')

    prec_macro = precision_score(y_test, y_pred, average='macro')
    rec_macro = recall_score(y_test, y_pred, average='macro')
    f1_s_macro = f1_score(y_test, y_pred, average='macro')

    prec_weight = precision_score(y_test, y_pred, average='weighted')
    rec_weight = recall_score(y_test, y_pred, average='weighted')
    f1_s_weight = f1_score(y_test, y_pred, average='weighted')


    classifi_report = classification_report(y_test, y_pred, target_names=target_names)

    # Update metrics
    self.accuracies.append(acc)
    self.losses.append(loss)
    self.prec_micros.append(prec_micro)
    self.rec_micro.append(rec_micro)
    self.f1_s_micro.append(f1_s_micro)
    self.prec_macro.append(prec_macro)
    self.rec_macro.append(rec_macro)
    self.f1_s_macro.append(f1_s_macro)
    self.prec_weight.append(prec_weight)
    self.rec_weight.append(rec_weight)
    self.f1_s_weight.append(f1_s_weight)

    if show:
      print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))
      print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
      print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
      print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

      print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
      print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
      print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

      print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
      print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
      print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

      print('\nClassification Report\n')
      print(classification_report(y_test, y_pred, target_names=target_names))

    return acc, loss, prec_micro, rec_micro, f1_s_micro, prec_macro, rec_macro, f1_s_macro, prec_weight, rec_weight, f1_s_weight, classifi_report

  def compute_confusion_matrix(self, y_pred, y_test):
    cm = confusion_matrix(y_true=y_test, y_pred = y_pred)
    #print(cm)
    # cm_display = ConfusionMatrixDisplay(cm).plot()
    self.confusion_matrices.append(cm)


  def send_to_drive(self, name : str):
    if not name:
      print("Error: Filename is empty")
      return
    
    df2 = pd.DataFrame(np.array([self.accuracies, self.prec_micros, self.rec_micro, self.f1_s_micro, self.prec_macro, self.rec_macro, self.f1_s_macro, self.prec_weight, self.rec_weight, self.f1_s_weight, self.times_per_epoch]),
    columns=['accuracies', 'prec_micros', 'rec_micro', 'f1_s_micro', 'prec_macro', 'rec_macro', 'f1_s_macro', 'prec_weight','rec_weight', 'f1_s_weight', 'times_per_epoch'])
    destination = "/content/drive/My Drive/" + name + ".xlsx"
    df2.to_excel(destination)
    print("Saved at: {}".format(destination))

    


spyketorch_metrics = Metrics()


Mounted at /content/drive/


In [2]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.end_time = time.time()
        self.execution_time = self.end_time - self.start_time

    def get_execution_time(self):
        return self.execution_time



In [3]:
# brian2 is a simulator for spiking neural networks. It allows to simulate neuraldinamics 
# using custom differential equations, but it is very computationally intensive.
# https://brian2.readthedocs.io/en/stable/resources/tutorials/index.html

!pip install brian2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.3 MB/s eta 0:00:00


In [4]:
from keras.datasets import mnist
from brian2 import *
import brian2.numpy_ as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# simplified classification (0 1 and 8)
X_train = X_train[(y_train == 1) | (y_train == 0) | (y_train == 8)]
y_train = y_train[(y_train == 1) | (y_train == 0) | (y_train == 8)]
X_test = X_test[(y_test == 1) | (y_test == 0) | (y_test == 8)]
y_test = y_test[(y_test == 1) | (y_test == 0) | (y_test == 8)]

# pixel intensity to Hz (255 becoms ~63Hz)
X_train = X_train / 4 
X_test = X_test / 4

X_train.shape, X_test.shape

11490434/11490434 [==============================] - 0s 0us/step


((18516, 28, 28), (3089, 28, 28))

In [ ]:
# plt.figure(figsize=(16,8))
# for img in range(32):
#     plt.subplot(4,8,1+img)
#     plt.title(y_train[img])
#     plt.imshow(X_train[img])
#     plt.axis('off')

In [7]:
n_input = 28*28 # input layer
n_e = 100 # e - excitatory
n_i = n_e # i - inhibitory

v_rest_e = -60.*mV # v - membrane potential
v_reset_e = -65.*mV
v_thresh_e = -52.*mV

v_rest_i = -60.*mV
v_reset_i = -45.*mV
v_thresh_i = -40.*mV

taupre = 20*ms
taupost = taupre
gmax = .05 #.01
dApre = .01
dApost = -dApre * taupre / taupost * 1.05
dApost *= gmax 
dApre *= gmax 

# Apre and Apost - presynaptic and postsynaptic traces, lr - learning rate
stdp='''w : 1
    lr : 1 (shared)
    dApre/dt = -Apre / taupre : 1 (event-driven)
    dApost/dt = -Apost / taupost : 1 (event-driven)'''
pre='''ge += w
    Apre += dApre
    w = clip(w + lr*Apost, 0, gmax)'''
post='''Apost += dApost
    w = clip(w + lr*Apre, 0, gmax)'''

class Model():
    
    def __init__(self, debug=False):
        app = {}
                
        # input images as rate encoded Poisson generators
        app['PG'] = PoissonGroup(n_input, rates=np.zeros(n_input)*Hz, name='PG')
        
        # excitatory group
        neuron_e = '''
            dv/dt = (ge*(0*mV-v) + gi*(-100*mV-v) + (v_rest_e-v)) / (100*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            dgi/dt = -gi / (10*ms) : 1
            '''
        app['EG'] = NeuronGroup(n_e, neuron_e, threshold='v>v_thresh_e', refractory=5*ms, reset='v=v_reset_e', method='euler', name='EG')
        app['EG'].v = v_rest_e - 20.*mV
        
        if (debug):
            app['ESP'] = SpikeMonitor(app['EG'], name='ESP')
            app['ESM'] = StateMonitor(app['EG'], ['v'], record=True, name='ESM')
            app['ERM'] = PopulationRateMonitor(app['EG'], name='ERM')
        
        # ibhibitory group
        neuron_i = '''
            dv/dt = (ge*(0*mV-v) + (v_rest_i-v)) / (10*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            '''
        app['IG'] = NeuronGroup(n_i, neuron_i, threshold='v>v_thresh_i', refractory=2*ms, reset='v=v_reset_i', method='euler', name='IG')
        app['IG'].v = v_rest_i - 20.*mV

        if (debug):
            app['ISP'] = SpikeMonitor(app['IG'], name='ISP')
            app['ISM'] = StateMonitor(app['IG'], ['v'], record=True, name='ISM')
            app['IRM'] = PopulationRateMonitor(app['IG'], name='IRM')
        
        # poisson generators one-to-all excitatory neurons with plastic connections 
        app['S1'] = Synapses(app['PG'], app['EG'], stdp, on_pre=pre, on_post=post, method='euler', name='S1')
        app['S1'].connect()
        app['S1'].w = 'rand()*gmax' # random weights initialisation
        app['S1'].lr = 1 # enable stdp        
        
        if (debug):
            # some synapses
            app['S1M'] = StateMonitor(app['S1'], ['w', 'Apre', 'Apost'], record=app['S1'][380,:4], name='S1M') 
        
        # excitatory neurons one-to-one inhibitory neurons
        app['S2'] = Synapses(app['EG'], app['IG'], 'w : 1', on_pre='ge += w', name='S2')
        app['S2'].connect(j='i')
        app['S2'].delay = 'rand()*10*ms'
        app['S2'].w = 3 # very strong fixed weights to ensure corresponding inhibitory neuron will always fire

        # inhibitory neurons one-to-all-except-one excitatory neurons
        app['S3'] = Synapses(app['IG'], app['EG'], 'w : 1', on_pre='gi += w', name='S3')
        app['S3'].connect(condition='i!=j')
        app['S3'].delay = 'rand()*5*ms'
        app['S3'].w = .03 # weights are selected in such a way as to maintain a balance between excitation and ibhibition
        
        self.net = Network(app.values())
        self.net.run(0*second)
        
    def __getitem__(self, key):
        return self.net[key]
    
    def train(self, X, epoch=1):        
        self.net['S1'].lr = 1 # stdp on
        
        for ep in range(epoch):
            for idx in range(len(X)):
                # active mode
                self.net['PG'].rates = X[idx].ravel()*Hz
                self.net.run(0.35*second)

                # passive mode
                self.net['PG'].rates = np.zeros(n_input)*Hz
                self.net.run(0.15*second)
        
    def evaluate(self, X):       
        self.net['S1'].lr = 0  # stdp off
        
        features = []
        for idx in range(len(X)):
            # rate monitor to count spikes
            mon = SpikeMonitor(self.net['EG'], name='RM')
            self.net.add(mon)
            
            # active mode
            self.net['PG'].rates = X[idx].ravel()*Hz
            self.net.run(0.35*second)
            
            # spikes per neuron foreach image
            features.append(np.array(mon.count, dtype=int8))
            
            # passive mode
            self.net['PG'].rates = np.zeros(n_input)*Hz
            self.net.run(0.15*second)
            
            self.net.remove(self.net['RM'])
            
        return features

In [ ]:
def plot_w(S1M):
    plt.rcParams["figure.figsize"] = (20,10)
    subplot(311)
    plot(S1M.t/ms, S1M.w.T/gmax)
    ylabel('w / wmax')
    subplot(312)
    plot(S1M.t/ms, S1M.Apre.T)
    ylabel('apre')
    subplot(313)
    plot(S1M.t/ms, S1M.Apost.T)
    ylabel('apost')
    tight_layout()
    show();
    
def plot_v(ESM, ISM, neuron=13):
    plt.rcParams["figure.figsize"] = (20,6)
    cnt = -50000 # tail
    plot(ESM.t[cnt:]/ms, ESM.v[neuron][cnt:]/mV, label='exc', color='r')
    plot(ISM.t[cnt:]/ms, ISM.v[neuron][cnt:]/mV, label='inh', color='b')
    plt.axhline(y=v_thresh_e/mV, color='pink', label='v_thresh_e')
    plt.axhline(y=v_thresh_i/mV, color='silver', label='v_thresh_i')
    legend()
    ylabel('v')
    show();
    
def plot_rates(ERM, IRM):
    plt.rcParams["figure.figsize"] = (20,6)
    plot(ERM.t/ms, ERM.smooth_rate(window='flat', width=0.1*ms)*Hz, color='r')
    plot(IRM.t/ms, IRM.smooth_rate(window='flat', width=0.1*ms)*Hz, color='b')
    ylabel('Rate')
    show();
    
def plot_spikes(ESP, ISP):
    plt.rcParams["figure.figsize"] = (20,6)
    plot(ESP.t/ms, ESP.i, '.r')
    plot(ISP.t/ms, ISP.i, '.b')
    ylabel('Neuron index')
    show();

def test0(train_items=30):
    '''
    STDP visualisation
    '''
    seed(0)
    
    model = Model(debug=True)
    model.train(X_train[:train_items], epoch=1)
    
    plot_w(model['S1M'])
    plot_v(model['ESM'], model['ISM'])
    plot_rates(model['ERM'], model['IRM'])
    plot_spikes(model['ESP'], model['ISP'])
    
test0()

In [10]:
def test1(train_items=5000, assign_items=1000, eval_items=1000):
    '''
    Feed train set to SNN with STDP
    Freeze STDP
    Feed train set to SNN again and collect generated features
    Train RandomForest on the top of these features and labels provided
    Feed test set to SNN and collect new features
    Predict labels with RandomForest and calculate accuacy score
    '''
    seed(0)
    
    model = Model()
    model.train(X_train[:train_items], epoch=1)
    model.net.store('train', 'train.b2')
    #model.net.restore('train', './train.b2')
    
    f_train = model.evaluate(X_train[:assign_items])
    clf = RandomForestClassifier(max_depth=4, random_state=0)
    clf.fit(f_train, y_train[:assign_items])
    print(clf.score(f_train, y_train[:assign_items]))

    f_test = model.evaluate(X_test[:eval_items])
    y_pred = clf.predict(f_test)
    print(accuracy_score(y_pred, y_test[:eval_items]))

    cm = confusion_matrix(y_pred, y_test[:eval_items])
    print(cm)
    
test1()

0.935
0.907
[[299   0  35]
 [  0 368  43]
 [  5  10 240]]


In [11]:
def test2(train_items=5000, assign_items=1000, eval_items=1000):
    '''
    Freeze STDP at start
    Feed train set to SNN and collect generated features
    Train RandomForest on the top of these features and labels provided
    Feed test set to SNN and collect new features
    Predict labels with RandomForest and calculate accuacy score
    '''
    seed(0)
    
    model = Model()
        
    f_train = model.evaluate(X_train[:assign_items])
    clf = RandomForestClassifier(max_depth=4, random_state=0)
    clf.fit(f_train, y_train[:assign_items])
    print(clf.score(f_train, y_train[:assign_items]))

    f_test = model.evaluate(X_test[:eval_items])
    y_pred = clf.predict(f_test)
    print(accuracy_score(y_pred, y_test[:eval_items]))

    cm = confusion_matrix(y_pred, y_test[:eval_items])
    print(cm)
    
test2()

0.849
0.787
[[216   2  72]
 [ 10 359  34]
 [ 78  17 212]]


In [12]:
def test3(train_items=5000, eval_items=1000):
    '''
    Train and evaluate RandomForest without SNN
    '''
    seed(0)
    
    clf = RandomForestClassifier(max_depth=4, random_state=0)
    
    train_features = X_train[:train_items].reshape(-1,28*28)
    clf.fit(train_features, y_train[:train_items])
    print(clf.score(train_features, y_train[:train_items]))
    
    test_features = X_test[:eval_items].reshape(-1,28*28)
    y_pred = clf.predict(test_features)
    print(accuracy_score(y_pred, y_test[:eval_items]))

    cm = confusion_matrix(y_pred, y_test[:eval_items])
    print(cm)
    
test3()

0.973
0.984
[[303   0   4]
 [  0 370   3]
 [  1   8 311]]
